# High CO2 comparison
In this notebook, we'll compare the results presented in [Eisenhut M et al. 2017](https://www.cell.com/molecular-plant/pdf/S1674-2052(16)30219-2.pdf) for the transition from high to ambient CO2 with our transition from ambient oxygen to high oxygen. These conditions are comaprable because they both involve increasing photorespuratory flux by increasing the ratio of oxygen:CO2 available to rubisco.